# 🔭 Optimal Sub-Exposure Time in Astrophotography

This notebook demonstrates how to estimate the optimal sub-exposure time based on your camera, telescope, and sky conditions, inspired by Dr. Robin Glover’s method.

See https://www.youtube.com/watch?v=ub1HjvlCJ5Y

Includes:
- Cooling temperature effect on dark current
- Optional direct measured sky background input

In [ ]:
from IPython.display import display, Markdown

camera_profiles = {
    "ASI2600MC Pro": {
        "gains": {
            0:   {"read_noise": 3.2, "full_well": 50000, "gain": 0.8},
            100: {"read_noise": 1.2, "full_well": 25000, "gain": 0.25},
        },
        "pixel_size": 3.76,        # µm
        "qe": 0.8,                 # quantum efficiency
        "dark_current_0C": 0.0025,  # e⁻/s at 0°C
        "temp_coeff": 0.5,        # 50% reduction per –5°C
    },
    "ASI2600MM Pro": {
        "gains": {
            0:   {"read_noise": 3.2, "full_well": 50000, "gain": 0.8},
            100: {"read_noise": 1.2, "full_well": 25000, "gain": 0.25},
        },
        "pixel_size": 3.76,
        "qe": 0.91,
        "dark_current_0C": 0.0025,
        "temp_coeff": 0.5,
    },
}

telescope_profiles = {
    "RedCat 71":           {"aperture_mm": 71, "focal_length_mm": 350, "f_ratio": 4.9},
    "Askar V 60/Reducer":  {"aperture_mm": 60, "focal_length_mm": 270, "f_ratio": 4.5},
    "Askar V 80/Reducer":  {"aperture_mm": 80, "focal_length_mm": 384, "f_ratio": 4.8},
    "Askar V 80/Extender": {"aperture_mm": 80, "focal_length_mm": 600, "f_ratio": 7.5},
}


In [ ]:
def sky_background_electron_rate(
    mag_per_arcsec2: float,
    camera_name: str,
    telescope_name: str,
    filter_bandwidth_nm: float = 300,
) -> float:
    """
    Convert sky brightness (mag/arcsec²) → electrons/sec/pixel.
    """
    tel = telescope_profiles[telescope_name]
    cam = camera_profiles[camera_name]
 
    e = 10**(0.4 * (21.7 - mag_per_arcsec2))
    # 3.837 is a constant derived from the conversion of mag/arcsec² to e⁻/s/pixel
    # 550 nm is the reference wavelength for the conversion
    # cam["qe"] is the quantum efficiency of the camera
    # cam["pixel_size"] is in µm, so we square it to get pixel area in µm²
    # tel["f_ratio"] is the focal ratio of the telescope
    # The formula calculates the expected electron rate based on these parameters.
    t = (3.837 * filter_bandwidth_nm / 550 * cam["qe"] * cam["pixel_size"]**2 / tel["f_ratio"]**2 * e)
    
    return round(100 * t) / 100

def calculate_optimal_exposure(
    camera_name: str,
    gain_setting: int,
    telescope_name: str,
    filter_bandwidth_nm: float = 300,
    sky_brightness: float = None,
    measured_sky_e_per_s: float = None,
    temp_c: float = 0,
    tolerated_noise_percentage: float = 5
) -> dict:
    """
    Auto‑computes pixel_scale and then calculates optimal sub‑exposure time.
    """
    # print(f"🔍 DEBUG: Starting calculation for {camera_name} @ gain {gain_setting} with {telescope_name}")
    # print(f"🔍 DEBUG: Input parameters - sky_brightness: {sky_brightness}, measured_sky_e_per_s: {measured_sky_e_per_s}, temp_c: {temp_c}, tolerated_noise_percentage: {tolerated_noise_percentage}")

    # look up profiles
    cam = camera_profiles[camera_name]
    gain_prof = cam["gains"][gain_setting]
    tel = telescope_profiles[telescope_name]
    
    # print(f"🔍 DEBUG: Camera profile loaded - pixel_size: {cam['pixel_size']}µm, qe: {cam['qe']}, dark_current_0C: {cam['dark_current_0C']} e⁻/s")
    # print(f"🔍 DEBUG: Gain profile loaded - read_noise: {gain_prof['read_noise']} e⁻, full_well: {gain_prof['full_well']} e⁻")
    # print(f"🔍 DEBUG: Telescope profile loaded - aperture: {tel['aperture_mm']}mm, focal_length: {tel['focal_length_mm']}mm")

    # 1) Dark current @ temp_c
    steps = (0 - temp_c) / 5
    DC = cam["dark_current_0C"] * (cam["temp_coeff"] ** steps)
    # print(f"🔍 DEBUG: Step 1 - Dark current calculation:")
    # print(f"           Temperature steps from 0°C to {temp_c}°C: {steps:.1f} steps")
    # print(f"           Dark current at {temp_c}°C: {DC:.6f} e⁻/s/pixel")

    # 2) Sky signal
    if measured_sky_e_per_s is not None:
        S = measured_sky_e_per_s
        # print(f"🔍 DEBUG: Step 2 - Using measured sky background: {S:.3f} e⁻/s/pixel")
    elif sky_brightness is not None:
        S = sky_background_electron_rate(sky_brightness, camera_name, telescope_name, filter_bandwidth_nm)
        # print(f"🔍 DEBUG: Step 2 - Sky signal calculation:")
        # print(f"           Sky brightness: {sky_brightness:.2f} mag/arcsec²")
        # print(f"           Calculated sky electrons: {S:.3f} e⁻/s/pixel")
    else:
        raise ValueError("Must provide sky_brightness or measured_sky_e_per_s")

    # 3) Compute t_opt
    RN = gain_prof["read_noise"]
    C_factor = 1/(((100+tolerated_noise_percentage)/100)**2-1)
    t_opt = C_factor * RN**2 / (S + DC)
    # print(f"🔍 DEBUG: Step 3 - Optimal exposure calculation:")
    # print(f"           Read noise: {RN:.2f} e⁻")
    # print(f"           C factor: {C_factor}")
    # print(f"           Total noise source (S + DC): {S + DC:.6f} e⁻/s")
    # print(f"           Optimal exposure time: {t_opt:.2f} seconds")

    # 4) Accumulated electrons & warnings
    e_sky  = S * t_opt
    e_dark = DC * t_opt
    e_tot  = e_sky + e_dark
    fw     = gain_prof["full_well"]

    # print(f"🔍 DEBUG: Step 4 - Accumulated electrons:")
    # print(f"           Sky electrons: {e_sky:.2f} e⁻")
    # print(f"           Dark electrons: {e_dark:.2f} e⁻")
    # print(f"           Total electrons: {e_tot:.2f} e⁻")
    # print(f"           Full well capacity: {fw} e⁻")
    # print(f"           Well fill percentage: {(e_tot/fw)*100:.1f}%")

    warns = []
    if e_tot > 0.8 * fw:
        warns.append(f"⚠️ Saturation risk: {e_tot:.0f} e⁻ > 80% of full well ({fw})")
    # if e_sky < snr_factor * RN**2:
    #     warns.append(f"⚠️ Sky noise < {snr_factor}× read noise")
    
    # print(f"🔍 DEBUG: Warnings: {warns if warns else 'None'}")
    # print("🔍 DEBUG: Calculation complete!\n")

    return {
        "camera": camera_name,
        "gain": gain_setting,
        "telescope": telescope_name,
        "dark_current_e_per_s": DC,
        "sky_e_per_s": S,
        "t_opt_s": t_opt,
        "e_sky": e_sky,
        "e_dark": e_dark,
        "e_total": e_tot,
        "full_well": fw,
        "warnings": warns
    }

def sky_brightness_from_bortle(bortle_class: int) -> float:
    """
    Return the typical sky brightness (mag/arcsec²) for a given Bortle scale class.
    
    Bortle Class → Sky Brightness (mag/arcsec²):
      1 → 21.9
      2 → 21.7
      3 → 21.5
      4 → 21.0
      5 → 20.5
      6 → 19.5
      7 → 18.0
      8 → 17.0
      9 → 16.0
    
    Args:
        bortle_class: Integer from 1 (excellent dark site) to 9 (inner-city sky).
    
    Returns:
        Typical sky brightness in mag/arcsec².
    
    Raises:
        ValueError: If bortle_class is not in 1–9.
    """
    mapping = {
        1: 21.9,
        2: 21.7,
        3: 21.5,
        4: 21.0,
        5: 20.5,
        6: 19.5,
        7: 18.0,
        8: 17.0,
        9: 16.0,
    }
    try:
        return mapping[bortle_class]
    except KeyError:
        raise ValueError("Bortle class must be an integer from 1 to 9") from None

In [101]:
def generate_filter_exposure_table(
    telescope_name: str,
    camera_name: str,
    gain_setting: int,
    temp_c: float = 0,
    tolerated_noise_percentage: float = 2,
    sky_brightness: float = None,
    measured_sky_e_per_s: float = None,
    custom_filters: dict = None
) -> None:
    """
    Generate a table with optimal exposure times for various common filter types.
    
    Args:
        telescope_name: Name of the telescope from telescope_profiles
        camera_name: Name of the camera from camera_profiles
        gain_setting: Gain setting to use
        temp_c: Camera temperature in Celsius
        tolerated_noise_percentage: Percentage of sky noise to tolerate (default: 2%)
        sky_brightness: Sky brightness in mag/arcsec² (optional)
        measured_sky_e_per_s: Directly measured sky background in e⁻/s/pixel (optional)
        custom_filters: Dictionary of custom filters to use instead of default filters
                        Format: {"Filter_Name": {"bandwidth_nm": value, "label": "Label", "type": "Type"}}
    """
    # Define default filter configurations with better organization by type
    default_filters = {
        # Broadband filters
        "L": {"bandwidth_nm": 300, "label": "Luminance", "type": "Broadband"},
        "R": {"bandwidth_nm": 100, "label": "Red", "type": "Broadband"},
        "G": {"bandwidth_nm": 100, "label": "Green", "type": "Broadband"},
        "B": {"bandwidth_nm": 100, "label": "Blue", "type": "Broadband"},
        
        # Narrowband filters
        "Hα": {"bandwidth_nm": 6, "label": "Hydrogen-α", "type": "Narrowband"},
        "OIII": {"bandwidth_nm": 6, "label": "Oxygen-III", "type": "Narrowband"},
        "SII": {"bandwidth_nm": 6, "label": "Sulfur-II", "type": "Narrowband"}
    }
    
    # Use custom filters if provided, otherwise use defaults
    filters = custom_filters if custom_filters is not None else default_filters
    
    # Calculate exposure time for each filter
    results = {}
    for filter_name, filter_info in filters.items():
        result = calculate_optimal_exposure(
            camera_name=camera_name,
            gain_setting=gain_setting,
            telescope_name=telescope_name,
            filter_bandwidth_nm=filter_info["bandwidth_nm"],
            sky_brightness=sky_brightness,
            measured_sky_e_per_s=measured_sky_e_per_s,
            temp_c=temp_c,
            tolerated_noise_percentage=tolerated_noise_percentage
        )
        results[filter_name] = result
    
    # Display setup information with better formatting
    display(Markdown(f"# Exposure Table: {telescope_name} + {camera_name}"))
    
    # Create setup info as a styled box
    setup_info = [
        f"### Configuration",
        f"- **Telescope**: {telescope_name} (f/{telescope_profiles[telescope_name]['f_ratio']})",
        f"- **Camera**: {camera_name} at Gain {gain_setting}",
        f"- **Temperature**: {temp_c}°C",
        f"- **Noise Tolerance**: {tolerated_noise_percentage}% of sky noise"
    ]
    
    # Add sky brightness or measured sky background info
    if sky_brightness is not None:
        bortle = 1 + int((22-sky_brightness)/0.8) if sky_brightness < 21.9 else 1
        setup_info.append(f"- **Sky Brightness**: {sky_brightness:.2f} mag/arcsec² (Bortle {bortle})")
    else:
        setup_info.append(f"- **Sky Background**: {measured_sky_e_per_s:.4f} e⁻/s/pixel")
    
    display(Markdown("\n".join(setup_info)))
    
    # Group results by filter type
    filter_types = {}
    
    for filter_name, result in results.items():
        t_opt = result["t_opt_s"]
        filter_type = filters[filter_name].get("type", "Other")
        
        # Format the time in appropriate units with better precision handling
        if t_opt < 1:
            formatted_time = f"{t_opt * 1000:.2f}ms"
        elif t_opt < 60:
            formatted_time = f"{t_opt:.2f}s"
        elif t_opt < 3600:
            minutes = int(t_opt // 60)
            seconds = round(t_opt % 60)
            formatted_time = f"{minutes}m {seconds:02d}s"
        else:
            hours = int(t_opt // 3600)
            minutes = int((t_opt % 3600) // 60)
            seconds = round(t_opt % 60)
            if seconds > 0:
                formatted_time = f"{hours}h {minutes:02d}m {seconds:02d}s"
            else:
                formatted_time = f"{hours}h {minutes:02d}m"
            
        # Calculate the well usage percentage
        well_usage = (result["e_total"] / result["full_well"]) * 100
        
        # Create a formatted result entry
        formatted_result = {
            "Filter": f"{filter_name}",
            "Description": f"{filters[filter_name]['label']} ({filters[filter_name]['bandwidth_nm']}nm)",
            "Optimal Exposure": formatted_time,
            "Sky Signal": f"{result['sky_e_per_s']:.3f} e⁻/s",
            "Well Usage": f"{well_usage:.1f}%",
            "Warnings": ", ".join(result["warnings"]) if result["warnings"] else None
        }
        
        # Add to appropriate filter type group
        if filter_type not in filter_types:
            filter_types[filter_type] = []
        filter_types[filter_type].append(formatted_result)
    
    # Display tables by filter type
    for filter_type, results_list in filter_types.items():
        display(Markdown(f"## {filter_type} Filters"))
        create_markdown_table(results_list)
    
    # Add a legend for warnings
    if any(result["warnings"] for result in results.values()):
        display(Markdown("### Warning Legend"))
        warning_md = "- ⚠️ **Sky noise < 10× read noise**: Exposure may be read noise limited\n"
        warning_md += "- ⚠️ **Well capacity > 80%**: Risk of non-linearity or saturation\n"
        warning_md += "- ⚠️ **Very short exposure**: May be below camera's minimum exposure time"
        display(Markdown(warning_md))

def create_markdown_table(results):
    """Helper function to create a markdown table from results"""
    if not results:
        display(Markdown("*No filters in this category*"))
        return
        
    table_rows = ["| Filter | Description | Optimal Exposure | Sky Signal | Well Usage | Warnings |", 
                 "|--------|-------------|------------------|------------|------------|----------|"]
    
    for row in results:
        warnings = row["Warnings"] if row["Warnings"] else ""
        table_rows.append(
            f"| {row['Filter']} | {row['Description']} | {row['Optimal Exposure']} | {row['Sky Signal']} | {row['Well Usage']} | {warnings} |"
        )
    
    display(Markdown("\n".join(table_rows)))


In [108]:
for telescope in telescope_profiles:
    generate_filter_exposure_table(
        telescope_name=telescope,
        camera_name="ASI2600MM Pro",
        gain_setting=100,
        temp_c=0,
        tolerated_noise_percentage=2,
        sky_brightness=sky_brightness_from_bortle(1))

    generate_filter_exposure_table(
        telescope_name=telescope,
        camera_name="ASI2600MC Pro",
        gain_setting=100,
        temp_c=0,
        tolerated_noise_percentage=2,
        sky_brightness=sky_brightness_from_bortle(1),
        custom_filters={
            "R": {"bandwidth_nm": 100, "label": "Red", "type": "Broadband"},
            "G": {"bandwidth_nm": 100, "label": "Green", "type": "Broadband"},
            "B": {"bandwidth_nm": 100, "label": "Blue", "type": "Broadband"},
            "Hα": {"bandwidth_nm": 10, "label": "Hydrogen-α", "type": "Narrowband"},
            "OIII": {"bandwidth_nm": 10, "label": "Oxygen-III", "type": "Narrowband"},
        })

# Exposure Table: RedCat 71 + ASI2600MM Pro

### Configuration
- **Telescope**: RedCat 71 (f/4.9)
- **Camera**: ASI2600MM Pro at Gain 100
- **Temperature**: 0°C
- **Noise Tolerance**: 2% of sky noise
- **Sky Brightness**: 21.90 mag/arcsec² (Bortle 1)

## Broadband Filters

| Filter | Description | Optimal Exposure | Sky Signal | Well Usage | Warnings |
|--------|-------------|------------------|------------|------------|----------|
| L | Luminance (300nm) | 38.22s | 0.930 e⁻/s | 0.1% |  |
| R | Red (100nm) | 1m 54s | 0.310 e⁻/s | 0.1% |  |
| G | Green (100nm) | 1m 54s | 0.310 e⁻/s | 0.1% |  |
| B | Blue (100nm) | 1m 54s | 0.310 e⁻/s | 0.1% |  |

## Narrowband Filters

...

### Configuration
- **Telescope**: Askar V 80/Extender (f/7.5)
- **Camera**: ASI2600MC Pro at Gain 100
- **Temperature**: 0°C
- **Noise Tolerance**: 2% of sky noise
- **Sky Brightness**: 21.90 mag/arcsec² (Bortle 1)

## Broadband Filters

| Filter | Description | Optimal Exposure | Sky Signal | Well Usage | Warnings |
|--------|-------------|------------------|------------|------------|----------|
| R | Red (100nm) | 4m 51s | 0.120 e⁻/s | 0.1% |  |
| G | Green (100nm) | 4m 51s | 0.120 e⁻/s | 0.1% |  |
| B | Blue (100nm) | 4m 51s | 0.120 e⁻/s | 0.1% |  |

## Narrowband Filters

| Filter | Description | Optimal Exposure | Sky Signal | Well Usage | Warnings |
|--------|-------------|------------------|------------|------------|----------|
| Hα | Hydrogen-α (10nm) | 47m 31s | 0.010 e⁻/s | 0.1% |  |
| OIII | Oxygen-III (10nm) | 47m 31s | 0.010 e⁻/s | 0.1% |  |